# Trying out `EvolvingGraphs`

We're interested in using the [`EvolvingGraphs`](http://evolvinggraphsjl.readthedocs.io/en/latest/overview.html).

In [1]:
Pkg.add("EvolvingGraphs")
Pkg.add("Interact")
Pkg.add("TikzGraphs")
Pkg.add("LightGraphs")

using EvolvingGraphs

INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of EvolvingGraphs
INFO: Use `Pkg.update()` to get the latest versions of your packages
INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of Interact
INFO: Use `Pkg.update()` to get the latest versions of your packages
INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of TikzGraphs
INFO: Use `Pkg.update()` to get the latest versions of your packages
INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of LightGraphs
INFO: Use `Pkg.update()` to get the latest versions of your packages


The [tutorial for the EvolvingGraphs](http://evolvinggraphsjl.readthedocs.io/en/latest/tutorial.html) package has some stuff we can work through.

Let's start by creating a graph with a couple of edges at different times:

In [2]:
graph = evolving_graph()

Directed EvolvingGraph (0 nodes, 0 static edges, 0 timestamps)

In [3]:
add_edge!(graph, 1, 2, 1)

Node(1)->Node(2) at time 1

In [4]:
timestamps(graph)

1-element Array{Int64,1}:
 1

In [5]:
add_edge!(graph, 1, 3, 2)
timestamps(graph)

2-element Array{Int64,1}:
 1
 2

In [6]:
edges(graph)

2-element Array{EvolvingGraphs.TimeEdge{EvolvingGraphs.Node{Int64},Int64},1}:
 Node(1)->Node(2) at time 1
 Node(1)->Node(3) at time 2

This graph is _directed_, which is perfect for our use case. We should be able to reconstruct simple SKOS hierarchies in this system. Let's try playing with weighted edges.

In [7]:
wg = weighted_evolving_graph(String, Int, Int)

Directed EvolvingGraph (0 nodes, 0 static edges, 0 timestamps)

In [8]:
# Assume edge weight 1 = foaf:knows
add_edge!(wg, "http://example.com/Moomin", "http://example.com/Moomin_Mama", 1, 1)
add_edge!(wg, "http://example.com/Moomin", "http://example.com/Moomin_Papa", 1, 1)
add_edge!(wg, "http://example.com/Moomin", "http://example.com/Moomin_Mama", 2, 1)
add_edge!(wg, "http://example.com/Moomin", "http://example.com/Moomin_Papa", 2, 1)
add_edge!(wg, "http://example.com/Moomin", "http://example.com/Snorkmaiden", 2, 1)
add_edge!(wg, "http://example.com/Moomin", "http://example.com/Moomin_Mama", 3, 1)
add_edge!(wg, "http://example.com/Moomin", "http://example.com/Moomin_Papa", 3, 1)
add_edge!(wg, "http://example.com/Moomin", "http://example.com/Snorkmaiden", 3, 1)
add_edge!(wg, "http://example.com/Moomin", "http://example.com/Snuffkin", 3, 1)
add_edge!(wg, "http://example.com/Moomin_Papa", "http://example.com/Snuffkin", 3, 1)
add_edge!(wg, "http://example.com/Moomin_Mama", "http://example.com/Snorkmaiden", 3, 1)

Node(http://example.com/Moomin_Mama)-1->Node(http://example.com/Snorkmaiden) at time 3

Each timestamp can be converted to an adjacency matrix.

In [9]:
matrix(wg, 1)

5×5 Array{Float64,2}:
 0.0  1.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

In [10]:
matrix(wg, 3)

5×5 Array{Float64,2}:
 0.0  1.0  1.0  1.0  1.0
 0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

Or to edge lists.

In [11]:
edges(wg, 1)

2-element Array{EvolvingGraphs.WeightedTimeEdge{EvolvingGraphs.Node{String},Int64,Int64},1}:
 Node(http://example.com/Moomin)-1->Node(http://example.com/Moomin_Mama) at time 1
 Node(http://example.com/Moomin)-1->Node(http://example.com/Moomin_Papa) at time 1

With a populated graph, we can visualize its evolution over time.

In [38]:
using Interact
using TikzGraphs
using LightGraphs

@manipulate for timestamp = 1:3
    es = edges(wg, timestamp)
    g  = DiGraph()
    add_edge!(g, source(es[1]), target(es[1]))
end

Interact.Options{:SelectionSlider,Int64}(Signal{Int64}(2, nactions=1),"timestamp",2,"2",Interact.OptionDict(DataStructures.OrderedDict("1"=>1,"2"=>2,"3"=>3),Dict(2=>"2",3=>"3",1=>"1")),Any[],Any[],true,"horizontal")

LoadError: MethodError: no method matching add_edge!(::LightGraphs.DiGraph, ::EvolvingGraphs.Node{String}, ::EvolvingGraphs.Node{String})[0m
Closest candidates are:
  add_edge!{V}([1m[31m::EvolvingGraphs.AbstractStaticGraph{V,E}[0m, ::V, ::V) at /home/tjohnson/.julia/v0.5/EvolvingGraphs/src/graphs/time_graph.jl:88
  add_edge!([1m[31m::EvolvingGraphs.AbstractStaticGraph{V,E}[0m, ::Any, ::Any) at /home/tjohnson/.julia/v0.5/EvolvingGraphs/src/graphs/time_graph.jl:92
  add_edge!{V,T,E<:EvolvingGraphs.TimeEdge{V,T}}([1m[31m::EvolvingGraphs.EvolvingGraph{V,T,E<:EvolvingGraphs.TimeEdge,I}[0m, ::Any, ::Any, [1m[31m::Any[0m) at /home/tjohnson/.julia/v0.5/EvolvingGraphs/src/graphs/evolving_graph.jl:245
  ...[0m